In [1]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

### Queries

In [2]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

In [3]:
%%time

# count of MG enabled LUs at wire center level
# 10/24/22 - add UNI ecd date
# 10/25/22 - add N. number per Jen request

mg_query = """

            SELECT  distinct
                    O.WIRE_CENTER_ID
                    ,O.UNI_POLYGONID
                    ,O.ADDRESS
                    ,O.HNO
                    ,O.H_SUFFIX
                    ,O.H_PRE
                    ,O.ST
                    ,O.ST_SUFFIX
                    ,O.ST_POST
                    ,O.UNIT
                    ,O.COM
                    ,O.STATE
                    ,O.ZIP_CODE
                    ,O.LU_LATITUDE
                    ,O.LU_LONGITUDE
                    ,O.UNI_ECD
                    ,O.UNI_NDSJOBNUMBER
                    ,O.NDSJOBID

                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND UNI_POLYGONID in ('1178832','1178694','1178144',
                                          '1179498','1179547','1179121','1179499',  --Phoenix AZ turned up 9/30/22
                                          '1179687','1179674', -- Golden CO 10/10/22
                                          '1180012', -- portland OR turned up 10/15/22
                                          '1179864', -- denver CO turned up 10/15/22
                                          '1179569','1179553','1179688', -- st paul and mn turned up 10/14/22
                                          '1179808', -- turned up 10/21/22
                                          '1179862','1179836','1179604','1179838','1179887','1180054','1180222', --turned up 10/21/22
                                          '1179157','1179574', -- turned up 10/21/22
                                          '1179976', -- turned up 10/27/22
                                          '1179583', '1180254', '1180028', -- turned up 10/28/22

--waiting turn-up
'1179774',
'1180111',
'1180117',
'1180086',
'1180265',
'1180107',
'1180288',
'1180327'


                                          
                                          ) --add in MG polygon IDs here


"""

mg_lu_ora = pd.read_sql(mg_query, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 266 ms
Wall time: 17.4 s


In [4]:
mg_lu_ora.columns
mg_lu_ora['UNI_POLYGONID'].value_counts()

1179808    677
1180028    443
1179864    437
1179862    436
1179836    413
1179569    395
1179604    377
1180117    370
1179157    366
1178144    342
1179976    337
1178694    336
1179688    326
1179838    320
1179498    319
1180222    307
1180054    295
1179547    284
1179553    259
1179121    247
1179574    238
1180288    228
1179887    222
1180111    215
1179499    212
1179774    200
1178832    196
1179687    183
1179583    166
1179674    158
1180265    140
1180086     99
1180012     74
1180254     71
1180327     47
1180107      9
Name: UNI_POLYGONID, dtype: int64

In [5]:
# future turn-ups
# df_1179808 =  mg_lu_ora[mg_lu_ora['UNI_POLYGONID'] == '1179808']  # 677
# df_1179862 =  mg_lu_ora[mg_lu_ora['UNI_POLYGONID'] == '1179862']  # 436
# df_1179836 =  mg_lu_ora[mg_lu_ora['UNI_POLYGONID'] == '1179836']  # 413
# df_1179604 =  mg_lu_ora[mg_lu_ora['UNI_POLYGONID'] == '1179604']  # 377
# df_1179157 =  mg_lu_ora[mg_lu_ora['UNI_POLYGONID'] == '1179157']  # 366
# df_1179838 =  mg_lu_ora[mg_lu_ora['UNI_POLYGONID'] == '1179838']  # 320
# df_1180222 =  mg_lu_ora[mg_lu_ora['UNI_POLYGONID'] == '1180222']  # 307
# df_1180054 =  mg_lu_ora[mg_lu_ora['UNI_POLYGONID'] == '1180054']  # 295
# df_1179574 =  mg_lu_ora[mg_lu_ora['UNI_POLYGONID'] == '1179574']  # 238
# df_1179887 =  mg_lu_ora[mg_lu_ora['UNI_POLYGONID'] == '1179887']  # 222

In [6]:
# export

import time

TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate +"_mg_polygon_addresses.xlsx"

with pd.ExcelWriter(excelfilename) as writer:
    mg_lu_ora.to_excel(writer, sheet_name='main addresses', index = False)
#     df_1179808.to_excel(writer, sheet_name='1179808', index = False)
#     df_1179862.to_excel(writer, sheet_name='1179862', index = False)
#     df_1179836.to_excel(writer, sheet_name='1179836', index = False)
#     df_1179604.to_excel(writer, sheet_name='1179604', index = False)
#     df_1179157.to_excel(writer, sheet_name='1179157', index = False)
#     df_1179838.to_excel(writer, sheet_name='1179838', index = False)
#     df_1180222.to_excel(writer, sheet_name='1180222', index = False)
#     df_1180054.to_excel(writer, sheet_name='1180054', index = False)
#     df_1179574.to_excel(writer, sheet_name='1179574', index = False)
#     df_1179887.to_excel(writer, sheet_name='1179887', index = False)
